In [1]:
import numpy as np
import torch
import sys
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torch.utils import data
from torchvision import transforms
from torchvision.datasets import MNIST


import matplotlib.pyplot as plt
import time
import zipfile
import tarfile
from PIL import Image



cuda = torch.cuda.is_available()
cuda
# import torch_xla
# import torch_xla.core.xla_model as xm
# cuda = xm.xla_device()
# cuda


True

In [0]:
VERSION = "20200220"  #@param ["20200220","nightly", "xrt==1.15.0"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python3 pytorch-xla-env-setup.py --version $VERSION


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3566  100  3566    0     0    640      0  0:00:05  0:00:05 --:--:--   878
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to 20200220 ...
Uninstalling torch-1.4.0:
Done updating TPU runtime: <Response [200]>
  Successfully uninstalled torch-1.4.0
Uninstalling torchvision-0.5.0:
  Successfully uninstalled torchvision-0.5.0
Copying gs://tpu-pytorch/wheels/torch-nightly+20200220-cp36-cp36m-linux_x86_64.whl...
/ [1 files][ 79.6 MiB/ 79.6 MiB]                                                
Operation completed over 1 objects/79.6 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200220-cp36-cp36m-linux_x86_64.whl...
\ [1 files][111.9 MiB/111.9 MiB]                                                
Operation completed over 1 objects/111.9 MiB.                      

In [0]:
!pip3 install torchvision


In [0]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
with zipfile.ZipFile('/content/drive/My Drive/11-785-hw2p2-s20.tgz.zip','r') as zp:
    print('Starting to extract......')
    zp.extractall()
    print('Done!')




Starting to extract......
Done!


In [4]:

tf = tarfile.open('11-785-hw2p2-s20.tgz')
print('extacting......')
tf.extractall()
print('Done!')


extacting......
Done!


In [5]:
with zipfile.ZipFile('11-785hw2p2-s20/train_data.zip','r') as train_zip:
    print('Starting to extract......')
    train_zip.extractall()
    print('Done!')

Starting to extract......
Done!


In [6]:
with zipfile.ZipFile('11-785hw2p2-s20/validation_classification.zip','r') as valid_zip:
    print('Starting to extract......')
    valid_zip.extractall()
    print('Done!')

Starting to extract......
Done!


In [7]:
with zipfile.ZipFile('11-785hw2p2-s20/test_classification.zip','r') as test_zip:
    print('Starting to extract......')
    test_zip.extractall()
    print('Done!')

Starting to extract......
Done!


In [0]:
import os

In [0]:
def parse_data(datadir):
    img_list = []
    ID_list = []
    for root, directories, filenames in os.walk(datadir):  #root: median/1
        for filename in filenames:
            if filename.endswith('.jpg'):
                filei = os.path.join(root, filename)
                img_list.append(filei)
                ID_list.append(root.split('/')[-1])

    # construct a dictionary, where key and value correspond to ID and target
    uniqueID_list = list(set(ID_list))
    class_n = len(uniqueID_list)
    target_dict = dict(zip(uniqueID_list, range(class_n)))
    label_list = [target_dict[ID_key] for ID_key in ID_list]

    print('{}\t\t{}\n{}\t\t{}'.format('#Images', '#Labels', len(img_list), len(set(label_list))))
    return img_list, label_list, class_n

In [10]:
img_list, label_list, class_n = parse_data('train_data/medium')

#Images		#Labels
822155		2300


In [11]:
print(img_list[1], label_list[1])

train_data/medium/2149/158.jpg 1362


In [0]:
from torch.utils.data import Dataset, DataLoader

In [0]:
class ImageDataset(Dataset):
    def __init__(self, file_list, target_list):
        self.file_list = file_list
        self.target_list = target_list
        self.n_class = len(list(set(target_list)))

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img = Image.open(self.file_list[index])
        img = torchvision.transforms.ToTensor()(img)
        label = self.target_list[index]
        return img, label

## imagefolder


## Residual Block
Resnet: https://arxiv.org/pdf/1512.03385.pdf

Here is a basic usage of shortcut in Resnet

In [0]:
class BasicBlock(nn.Module):

    def __init__(self, channel_size, pad, stride):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(channel_size, channel_size, kernel_size=3, stride=stride, padding=pad, bias=False)
        self.bn1 = nn.BatchNorm2d(channel_size)
        self.shortcut = nn.Conv2d(channel_size, channel_size, kernel_size=1, stride=stride, bias=False)


    def forward(self, x):
        # print(x.size())
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn1(self.conv1(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [0]:
class Network(nn.Module):
    def __init__(self, num_feats, hidden_sizes, num_classes, feat_dim):
        super(Network, self).__init__()
        
        self.hidden_sizes = [num_feats] + hidden_sizes + [num_classes]
        
        self.layers = []

        for idx, channel_size in enumerate(hidden_sizes):
            if idx == 0 or idx == 1:
              self.layers.append(nn.Conv2d(in_channels=self.hidden_sizes[idx], 
                                          out_channels=self.hidden_sizes[idx+1], 
                                          kernel_size=5, stride=1, bias=False))
              self.layers.append(nn.BatchNorm2d(channel_size))
              self.layers.append(nn.ReLU(inplace=True))
              self.layers.append(nn.MaxPool2d(3,1,1))
              
            else:
              self.layers.append(nn.Conv2d(in_channels=self.hidden_sizes[idx], 
                                          out_channels=self.hidden_sizes[idx+1], 
                                          kernel_size=3, stride=1, bias=False))
              self.layers.append(nn.BatchNorm2d(channel_size))
              self.layers.append(nn.ReLU(inplace=True))
              
        self.layers.append(nn.MaxPool2d(3,1,1))
        self.layers.append(nn.Dropout(p=0.5, inplace=False))
        self.layers = nn.Sequential(*self.layers)
        self.linear_label = nn.Linear(self.hidden_sizes[-2], self.hidden_sizes[-1], bias=False)
        
        # For creating the embedding to be passed into the Center Loss criterion
        self.linear_closs = nn.Linear(self.hidden_sizes[-2], feat_dim, bias=False)
        self.relu_closs = nn.ReLU(inplace=True)
    
    def forward(self, x, evalMode=False):
        output = x
        output = self.layers(output)
            
        output = F.avg_pool2d(output, [output.size(2), output.size(3)], stride=1)
        output = output.reshape(output.shape[0], output.shape[1])
        
        label_output = self.linear_label(output)
        label_output = label_output/torch.norm(self.linear_label.weight, dim=1)
        
        # Create the feature embedding for the Center Loss
        closs_output = self.linear_closs(output)
        closs_output = self.relu_closs(closs_output)

        return closs_output, label_output

def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)

In [0]:
# class Network(nn.Module):
#     def __init__(self, num_feats, hidden_sizes, num_classes, feat_dim):
#         super(Network, self).__init__()
        
#         self.hidden_sizes = [num_feats] + hidden_sizes + [num_classes]
        
#         self.layers = []

#         for idx, channel_size in enumerate(hidden_sizes):
#             if idx == 0 or idx == 1:
#               self.layers.append(nn.Conv2d(in_channels=self.hidden_sizes[idx], 
#                                           out_channels=self.hidden_sizes[idx+1], 
#                                           kernel_size=5, stride=1, bias=False))
#               self.layers.append(nn.BatchNorm2d(channel_size))
#               self.layers.append(nn.ReLU(inplace=True))
              
#             else:
#               self.layers.append(nn.Conv2d(in_channels=self.hidden_sizes[idx], 
#                                           out_channels=self.hidden_sizes[idx+1], 
#                                           kernel_size=3, stride=1, bias=False))
#               self.layers.append(nn.BatchNorm2d(channel_size))
#               self.layers.append(nn.ReLU(inplace=True))
              
            
#         self.layers = nn.Sequential(*self.layers)
#         self.linear_label = nn.Linear(self.hidden_sizes[-2], self.hidden_sizes[-1], bias=False)
        
#         # For creating the embedding to be passed into the Center Loss criterion
#         self.linear_closs = nn.Linear(self.hidden_sizes[-2], feat_dim, bias=False)
#         self.relu_closs = nn.ReLU(inplace=True)
    
#     def forward(self, x, evalMode=False):
#         output = x
#         output = self.layers(output)
            
#         output = F.avg_pool2d(output, [output.size(2), output.size(3)], stride=1)
#         output = output.reshape(output.shape[0], output.shape[1])
        
#         label_output = self.linear_label(output)
#         label_output = label_output/torch.norm(self.linear_label.weight, dim=1)
        
#         # Create the feature embedding for the Center Loss
#         closs_output = self.linear_closs(output)
#         closs_output = self.relu_closs(closs_output)

#         return closs_output, label_output

# def init_weights(m):
#     if type(m) == nn.Conv2d or type(m) == nn.Linear:
#         torch.nn.init.xavier_normal_(m.weight.data)

## Trainnnig 

In [0]:
train_dataset = torchvision.datasets.ImageFolder(root='train_data/medium', 
                                                 transform=torchvision.transforms.ToTensor())
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, 
                                               shuffle=True, num_workers=8)

dev_dataset = torchvision.datasets.ImageFolder(root='validation_classification/medium', 
                                               transform=torchvision.transforms.ToTensor())
dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=32, 
                                             shuffle=True, num_workers=8)

## Layers for Architecture


In [0]:
numEpochs = 50
num_feats = 3

learningRate = 0.01
weightDecay = 5e-4

# hidden_sizes = [32,32,32,32,64, 64, 64, 64, 128, 128, 128, 128, 256, 256]
# hidden_sizes = [32, 64, 128, 128, 256, 512]
hidden_sizes = [32, 64, 192, 384, 512, 256]
num_classes = len(train_dataset.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = xm.xla_device()

## Center Loss

In [0]:
class CenterLoss(nn.Module):
    """
    Args:
        num_classes (int): number of classes.
        feat_dim (int): feature dimension.
    """
    def __init__(self, num_classes, feat_dim, device=torch.device('cpu')):
        super(CenterLoss, self).__init__()
        self.num_classes = num_classes
        self.feat_dim = feat_dim
        self.device = device
        
        self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim).to(self.device))

    def forward(self, x, labels):
        """
        Args:
            x: feature matrix with shape (batch_size, feat_dim).
            labels: ground truth labels with shape (batch_size).
        """
        batch_size = x.size(0)
        distmat = torch.pow(x, 2).sum(dim=1, keepdim=True).expand(batch_size, self.num_classes) + \
                  torch.pow(self.centers, 2).sum(dim=1, keepdim=True).expand(self.num_classes, batch_size).t()
        distmat.addmm_(1, -2, x, self.centers.t())

        classes = torch.arange(self.num_classes).long().to(self.device)
        labels = labels.unsqueeze(1).expand(batch_size, self.num_classes)
        mask = labels.eq(classes.expand(batch_size, self.num_classes))

        dist = []
        for i in range(batch_size):
            value = distmat[i][mask[i]]
            value = value.clamp(min=1e-12, max=1e+12) # for numerical stability
            dist.append(value)
        dist = torch.cat(dist)
        loss = dist.mean()

        return loss

In [0]:
def train_closs(scheduler_closs, scheduler_label,model, data_loader, test_loader, task='Classification'):
    model.train()
    global optimizer_closs
    global optimizer_label
    for epoch in range(numEpochs):
        # if epoch == 1:
        #   optimizer_closs = torch.optim.SGD(criterion_closs.parameters(), lr=0.1)
        #   optimizer_label = torch.optim.SGD(network.parameters(), lr=0.007, weight_decay=weightDecay, momentum=0.9)
        # if epoch == 2:
        #   optimizer_closs = torch.optim.Adam(criterion_closs.parameters(),lr = 0.0001)
        #   optimizer_label = torch.optim.Adam(network.parameters(), lr = 0.0001)
        # if epoch == 3:
        #   optimizer_closs = torch.optim.SGD(criterion_closs.parameters(), lr=0.03)
        #   optimizer_label = torch.optim.SGD(network.parameters(), lr=0.003, weight_decay=weightDecay, momentum=0.9)
        # if epoch == 4:
        #   optimizer_closs = torch.optim.SGD(criterion_closs.parameters(), lr=0.01)
        #   optimizer_label = torch.optim.SGD(network.parameters(), lr=0.001, weight_decay=weightDecay, momentum=0.9)
        print('closs lr: ', optimizer_closs.param_groups[0]['lr'])
        print('label lr: ', optimizer_label.param_groups[0]['lr'])
        avg_loss = 0.0
        start_time = time.time()
        for batch_num, (feats, labels) in enumerate(data_loader):
            feats, labels = feats.to(device), labels.to(device)
            
            optimizer_label.zero_grad()
            optimizer_closs.zero_grad()
            
            feature, outputs = model(feats)
            # print('feature size: ', feature.size())
            # print('outputs size: ', outputs.size())

            l_loss = criterion_label(outputs, labels.long())
            c_loss = criterion_closs(feature, labels.long())
            loss = l_loss + closs_weight * c_loss
            
            loss.backward()
            
            optimizer_label.step()
            # by doing so, weight_cent would not impact on the learning of centers
            for param in criterion_closs.parameters():
                param.grad.data *= (1. / closs_weight)
            optimizer_closs.step()
            
            avg_loss += loss.item()

            if batch_num % 50 == 49:
                print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, batch_num+1, avg_loss/50))
                avg_loss = 0.0    
            
            # torch.cuda.empty_cache()
            del feats
            del labels
            del loss
        scheduler_closs.step()
        scheduler_label.step()
        end_time = time.time()
        print('Epoch: ', epoch + 1, 'Time: ', end_time - start_time, 's')
        if task == 'Classification':
            torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_closs_state_dict': optimizer_closs.state_dict(),
            'optimizer_closs_state_dict': optimizer_label.state_dict(),
            'c_loss': c_loss,
            'l_loss': l_loss,
            'scheduler_closs': scheduler_closs,
            'scheduler_closs': scheduler_label,
            }, '/content/drive/My Drive/train_valid_exp4-epoch{}.pth'.format(epoch+1))
            val_loss, val_acc, vc_loss, vl_loss = test_classify_closs(model, test_loader)
            train_loss, train_acc, tc_loss, tl_loss = test_classify_closs(model, data_loader)
            print('Train Loss: {:.4f}\tTrain Accuracy: {:.4f}\tVal Loss: {:.4f}\tVal Accuracy: {:.4f}'.
                  format(train_loss, train_acc, val_loss, val_acc))
        else:
            test_verify(model, test_loader)


def test_classify_closs(model, test_loader):
    model.eval()
    test_loss = []
    accuracy = 0
    total = 0
    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)
        feature, outputs = model(feats)
        
        _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
        pred_labels = pred_labels.view(-1)
        
        l_loss = criterion_label(outputs, labels.long())
        c_loss = criterion_closs(feature, labels.long())
        loss = l_loss + closs_weight * c_loss
        
        accuracy += torch.sum(torch.eq(pred_labels, labels)).item()
        total += len(labels)
        test_loss.extend([loss.item()]*feats.size()[0])
        del feats
        del labels

    model.train()
    return np.mean(test_loss), accuracy/total, c_loss, l_loss

In [0]:
closs_weight = 1
lr_cent = 0.5
feat_dim = 100

network = Network(num_feats, hidden_sizes, num_classes, feat_dim)
network.apply(init_weights)

criterion_label = nn.CrossEntropyLoss()
criterion_closs = CenterLoss(num_classes, feat_dim, device)
optimizer_label = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
optimizer_closs = torch.optim.SGD(criterion_closs.parameters(), lr=lr_cent)
# optimizer_label = torch.optim.Adam(network.parameters())
# optimizer_closs = torch.optim.Adam(criterion_closs.parameters())
# scheduler_closs = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_closs, mode='min', factor=0.1, patience=1)
# scheduler_label = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_label, mode='min', factor=0.1, patience=1)
scheduler_closs = torch.optim.lr_scheduler.StepLR(optimizer_closs, step_size=1,gamma=0.8)
scheduler_label = torch.optim.lr_scheduler.StepLR(optimizer_label, step_size=1,gamma=0.8)


## loading model

In [0]:
# network = Network(num_feats, hidden_sizes, num_classes, feat_dim)
# checkpoint = torch.load('/content/drive/My Drive/train_valid_exp4-epoch16.pth')
# network.load_state_dict(checkpoint['model_state_dict'])
# network.to(device)
# network.eval()

In [22]:
print(network)

Network(
  (layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), bias=False)
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (9): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (12): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): Conv2d(384, 512, kernel_size=

In [0]:
network.train()
network.to(device)
train_closs(scheduler_closs,scheduler_label,network, train_dataloader, dev_dataloader)

closs lr:  0.5
label lr:  0.01
Epoch: 1	Batch: 50	Avg-Loss: 107.3735
Epoch: 1	Batch: 100	Avg-Loss: 101.2362
Epoch: 1	Batch: 150	Avg-Loss: 97.1800
Epoch: 1	Batch: 200	Avg-Loss: 92.9629
Epoch: 1	Batch: 250	Avg-Loss: 89.0496
Epoch: 1	Batch: 300	Avg-Loss: 86.0650
Epoch: 1	Batch: 350	Avg-Loss: 82.3832
Epoch: 1	Batch: 400	Avg-Loss: 79.0124
Epoch: 1	Batch: 450	Avg-Loss: 75.5803
Epoch: 1	Batch: 500	Avg-Loss: 72.9597
Epoch: 1	Batch: 550	Avg-Loss: 69.7090
Epoch: 1	Batch: 600	Avg-Loss: 67.1493
Epoch: 1	Batch: 650	Avg-Loss: 64.5404
Epoch: 1	Batch: 700	Avg-Loss: 61.7944
Epoch: 1	Batch: 750	Avg-Loss: 59.9817
Epoch: 1	Batch: 800	Avg-Loss: 58.3144
Epoch: 1	Batch: 850	Avg-Loss: 54.8805
Epoch: 1	Batch: 900	Avg-Loss: 53.2664
Epoch: 1	Batch: 950	Avg-Loss: 50.9685
Epoch: 1	Batch: 1000	Avg-Loss: 49.5680
Epoch: 1	Batch: 1050	Avg-Loss: 47.8803
Epoch: 1	Batch: 1100	Avg-Loss: 46.3071
Epoch: 1	Batch: 1150	Avg-Loss: 44.4985
Epoch: 1	Batch: 1200	Avg-Loss: 42.4051
Epoch: 1	Batch: 1250	Avg-Loss: 40.9843
Epoch: 1	Bat

In [27]:
print('closs lr: ', optimizer_closs.param_groups[0]['lr'])

closs lr:  0.039883221538436285


In [0]:
for name, param in network.named_parameters():
    if param.requires_grad:
        print (name, param.data)

## Start to load test data and do the test

In [0]:
def get_pred_list(model, test_loader):
    model.eval()
    test_loss = []
    accuracy = 0
    total = 0
    pred_list = []
    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)
        feature, outputs = model(feats)
        
        _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
        pred_labels = pred_labels.view(-1)
        pred_list.append(pred_labels)

    model.train()
    return pred_list

In [50]:
my_model = Network(num_feats, hidden_sizes, num_classes, feat_dim)
checkpoint = torch.load('/content/drive/My Drive/train_valid_exp4-epoch3.pth')
my_model.load_state_dict(checkpoint['model_state_dict'])
my_model.to(device)
my_model.eval()


Network(
  (layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), bias=False)
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (9): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (12): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): Conv2d(384, 256, kernel_size=

In [0]:
test_dataset = torchvision.datasets.ImageFolder(root='test_classification', 
                                                 transform=torchvision.transforms.ToTensor())
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, 
                                               shuffle=False, num_workers=8)

In [52]:
print(train_dataset.class_to_idx)


{'0': 0, '1': 1, '10': 2, '100': 3, '1000': 4, '1001': 5, '1002': 6, '1003': 7, '1004': 8, '1005': 9, '1006': 10, '1007': 11, '1008': 12, '1009': 13, '101': 14, '1010': 15, '1011': 16, '1012': 17, '1013': 18, '1014': 19, '1015': 20, '1016': 21, '1017': 22, '1018': 23, '1019': 24, '102': 25, '1020': 26, '1021': 27, '1022': 28, '1023': 29, '1024': 30, '1025': 31, '1026': 32, '1027': 33, '1028': 34, '1029': 35, '103': 36, '1030': 37, '1031': 38, '1032': 39, '1033': 40, '1034': 41, '1035': 42, '1036': 43, '1037': 44, '1038': 45, '1039': 46, '104': 47, '1040': 48, '1041': 49, '1042': 50, '1043': 51, '1044': 52, '1045': 53, '1046': 54, '1047': 55, '1048': 56, '1049': 57, '105': 58, '1050': 59, '1051': 60, '1052': 61, '1053': 62, '1054': 63, '1055': 64, '1056': 65, '1057': 66, '1058': 67, '1059': 68, '106': 69, '1060': 70, '1061': 71, '1062': 72, '1063': 73, '1064': 74, '1065': 75, '1066': 76, '1067': 77, '1068': 78, '1069': 79, '107': 80, '1070': 81, '1071': 82, '1072': 83, '1073': 84, '1074

In [53]:
print(dev_dataset.class_to_idx['1625'])

697


In [54]:


print('test data loading....')
pred_list = []
pred_list = get_pred_list(my_model, test_dataloader)
# pred_list = get_pred_list(network, test_dataloader)

print('test data GET!!')
print(pred_list)
# print(test_dataset.size)

test data loading....
test data GET!!
[tensor([ 704, 2199,  990,  401,  871, 2167, 1654, 1609, 2019, 1355,  378,  944,
        1694, 2111, 1657, 1497,  280, 1299,  907, 1265, 2245,  520,  104, 1739,
        1195, 1869,  274, 1455,   83,  747, 1118, 1661], device='cuda:0'), tensor([1353, 1031,  244, 1265,   99,  483,  340,  710, 1249, 1507, 1263, 1851,
        2170,  393, 1317,  128,  274,  871,  291, 2266, 2078,  662, 1950, 1095,
        1833, 2198, 1146, 1227,  965, 2108, 1022,  884], device='cuda:0'), tensor([1209,  190, 1285,  303, 1405,  846, 1689, 1012,  147, 1293, 1357, 1869,
        2264, 1550, 2122,  483, 1319, 1485, 1652,  957, 1351, 2190,    3, 1635,
         330,  874, 1669, 1482, 1492, 1687, 1898, 1483], device='cuda:0'), tensor([ 133, 2079, 1217, 1252,  338, 1443, 1168, 1824, 1876,  375,  479, 2100,
         627,  950,  542, 2154, 1912, 1840,  424, 1906, 1053,  826, 1549,  653,
        2061, 1538, 2022, 1099, 2266,   95,  551, 2080], device='cuda:0'), tensor([2270, 1186, 2

In [55]:
idx = []
count = 5000
for i in range(4600):
  idx.append(str(count) + '.jpg')
  count+=1


pred = []
for i in pred_list:
    pred += i.cpu().tolist()
    
print(len(pred))
print(pred)

4600
[704, 2199, 990, 401, 871, 2167, 1654, 1609, 2019, 1355, 378, 944, 1694, 2111, 1657, 1497, 280, 1299, 907, 1265, 2245, 520, 104, 1739, 1195, 1869, 274, 1455, 83, 747, 1118, 1661, 1353, 1031, 244, 1265, 99, 483, 340, 710, 1249, 1507, 1263, 1851, 2170, 393, 1317, 128, 274, 871, 291, 2266, 2078, 662, 1950, 1095, 1833, 2198, 1146, 1227, 965, 2108, 1022, 884, 1209, 190, 1285, 303, 1405, 846, 1689, 1012, 147, 1293, 1357, 1869, 2264, 1550, 2122, 483, 1319, 1485, 1652, 957, 1351, 2190, 3, 1635, 330, 874, 1669, 1482, 1492, 1687, 1898, 1483, 133, 2079, 1217, 1252, 338, 1443, 1168, 1824, 1876, 375, 479, 2100, 627, 950, 542, 2154, 1912, 1840, 424, 1906, 1053, 826, 1549, 653, 2061, 1538, 2022, 1099, 2266, 95, 551, 2080, 2270, 1186, 2170, 993, 2013, 407, 1221, 1097, 597, 1584, 2150, 1848, 511, 151, 1506, 707, 2010, 1603, 78, 336, 1714, 877, 1130, 1484, 457, 1804, 1131, 1860, 2166, 22, 2087, 2290, 93, 1345, 1039, 1345, 1843, 715, 1765, 1424, 1836, 786, 1879, 2130, 1706, 1522, 51, 817, 1009, 0, 1

In [56]:
mapped = []
dict_test = train_dataset.class_to_idx
inv_map = {v: k for k, v in dict_test.items()}
for i in pred:
  mapped.append(int(inv_map[i]))
print(len(mapped))
print(mapped)

4600
[1631, 908, 189, 1359, 1782, 88, 417, 377, 746, 2217, 1338, 1848, 453, 829, 42, 276, 125, 2167, 1814, 2136, 95, 1466, 1091, 494, 2073, 610, 1244, 238, 1072, 1670, 2003, 423, 2215, 1926, 1217, 2136, 1087, 1432, 1303, 1637, 2121, 285, 2134, 595, 882, 1351, 2183, 1112, 1244, 1782, 126, 969, 8, 1594, 684, 1984, 579, 907, 2029, 2101, 1867, 826, 1918, 1794, 2086, 1169, 2154, 1270, 2262, 176, 449, 1909, 113, 2161, 2219, 610, 967, 323, 839, 1432, 2185, 265, 415, 186, 2213, 90, 100, 40, 1295, 1785, 430, 262, 271, 447, 637, 263, 1117, 80, 2093, 2124, 1301, 2297, 2049, 570, 617, 1335, 1429, 819, 1562, 1853, 1486, 868, 65, 585, 138, 644, 1946, 1741, 322, 1586, 784, 312, 749, 1988, 969, 1083, 1494, 800, 972, 2065, 882, 1892, 740, 1364, 2097, 1986, 1535, 354, 864, 592, 1458, 1133, 284, 1634, 738, 371, 1068, 130, 471, 1788, 2014, 264, 1409, 552, 2015, 602, 879, 1017, 807, 990, 1081, 2208, 1933, 2208, 588, 1641, 517, 228, 581, 1705, 62, 846, 464, 299, 1043, 1733, 1906, 0, 201, 1465, 2293, 1521, 2

In [57]:
print(idx)
print(len(idx))

['5000.jpg', '5001.jpg', '5002.jpg', '5003.jpg', '5004.jpg', '5005.jpg', '5006.jpg', '5007.jpg', '5008.jpg', '5009.jpg', '5010.jpg', '5011.jpg', '5012.jpg', '5013.jpg', '5014.jpg', '5015.jpg', '5016.jpg', '5017.jpg', '5018.jpg', '5019.jpg', '5020.jpg', '5021.jpg', '5022.jpg', '5023.jpg', '5024.jpg', '5025.jpg', '5026.jpg', '5027.jpg', '5028.jpg', '5029.jpg', '5030.jpg', '5031.jpg', '5032.jpg', '5033.jpg', '5034.jpg', '5035.jpg', '5036.jpg', '5037.jpg', '5038.jpg', '5039.jpg', '5040.jpg', '5041.jpg', '5042.jpg', '5043.jpg', '5044.jpg', '5045.jpg', '5046.jpg', '5047.jpg', '5048.jpg', '5049.jpg', '5050.jpg', '5051.jpg', '5052.jpg', '5053.jpg', '5054.jpg', '5055.jpg', '5056.jpg', '5057.jpg', '5058.jpg', '5059.jpg', '5060.jpg', '5061.jpg', '5062.jpg', '5063.jpg', '5064.jpg', '5065.jpg', '5066.jpg', '5067.jpg', '5068.jpg', '5069.jpg', '5070.jpg', '5071.jpg', '5072.jpg', '5073.jpg', '5074.jpg', '5075.jpg', '5076.jpg', '5077.jpg', '5078.jpg', '5079.jpg', '5080.jpg', '5081.jpg', '5082.jpg', '50

##  Write all data to the dataframe and to a csv file

In [0]:
import pandas as pd

In [59]:
df = pd.DataFrame({'Id':idx,'Category':mapped})
print(df.head())
df.to_csv('submission.csv', index = False)
# torch.save(network.state_dict(), '/content/drive/My Drive/model.pth')

         Id  Category
0  5000.jpg      1631
1  5001.jpg       908
2  5002.jpg       189
3  5003.jpg      1359
4  5004.jpg      1782


## 祝我成功 closs lr:  0.04431469059826254
label lr:  0.0008862938119652509

In [39]:
0.04431469059826254/2

0.02215734529913127

In [0]:
train_dataset = torchvision.datasets.ImageFolder(root='train_data/medium', 
                                                 transform=torchvision.transforms.ToTensor())
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, 
                                               shuffle=True, num_workers=8)

# dev_dataset = torchvision.datasets.ImageFolder(root='validation_classification/medium', 
#                                                transform=torchvision.transforms.ToTensor())
# dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=32, 
#                                              shuffle=True, num_workers=8)

In [0]:
optimizer_label = torch.optim.SGD(network.parameters(), lr=0.00044314690598262546, weight_decay=weightDecay, momentum=0.9)
optimizer_closs = torch.optim.SGD(criterion_closs.parameters(), lr=0.02215734529913127)
scheduler_closs = torch.optim.lr_scheduler.StepLR(optimizer_closs, step_size=1,gamma=0.8)
scheduler_label = torch.optim.lr_scheduler.StepLR(optimizer_label, step_size=1,gamma=0.8)

In [0]:
network.train()
network.to(device)
train_closs(scheduler_closs,scheduler_label,network, train_dataloader, dev_dataloader)

Streaming output truncated to the last 5000 lines.
Epoch: 10	Batch: 8450	Avg-Loss: 0.2331
Epoch: 10	Batch: 8500	Avg-Loss: 0.2646
Epoch: 10	Batch: 8550	Avg-Loss: 0.2460
Epoch: 10	Batch: 8600	Avg-Loss: 0.2710
Epoch: 10	Batch: 8650	Avg-Loss: 0.2435
Epoch: 10	Batch: 8700	Avg-Loss: 0.2282
Epoch: 10	Batch: 8750	Avg-Loss: 0.2522
Epoch: 10	Batch: 8800	Avg-Loss: 0.2432
Epoch: 10	Batch: 8850	Avg-Loss: 0.2521
Epoch: 10	Batch: 8900	Avg-Loss: 0.2711
Epoch: 10	Batch: 8950	Avg-Loss: 0.2406
Epoch: 10	Batch: 9000	Avg-Loss: 0.2761
Epoch: 10	Batch: 9050	Avg-Loss: 0.2519
Epoch: 10	Batch: 9100	Avg-Loss: 0.2383
Epoch: 10	Batch: 9150	Avg-Loss: 0.2389
Epoch: 10	Batch: 9200	Avg-Loss: 0.2587
Epoch: 10	Batch: 9250	Avg-Loss: 0.2660
Epoch: 10	Batch: 9300	Avg-Loss: 0.2327
Epoch: 10	Batch: 9350	Avg-Loss: 0.2373
Epoch: 10	Batch: 9400	Avg-Loss: 0.2561
Epoch: 10	Batch: 9450	Avg-Loss: 0.2468
Epoch: 10	Batch: 9500	Avg-Loss: 0.2498
Epoch: 10	Batch: 9550	Avg-Loss: 0.2675
Epoch: 10	Batch: 9600	Avg-Loss: 0.2237
Epoch: 10	Bat